In [1]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables):
#     print 'BranchedVariables',BranchedVariables
    unbranched=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)
    NodeNum=n-len(unbranched)
    
    
    for i in unbranched:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in unbranched: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in unbranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in unbranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in unbranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     print 'unbranched',unbranched
    return c, d,unbranched
#     print c, d,unbranched

In [2]:
def SecondModel (cc,dd,NotBranched):
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    ConstNum=numrows/len(NotBranched)
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))

    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in NotBranched: 
                indj=NotBranched.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
#     print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
    lenNotbranched=len(NotBranched)
   
    
    for t in range(lenNotbranched):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in NotBranched and i > NotBranched[t]:
                indx=NotBranched.index(i)
                indy=NotBranched.index(NotBranched[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,NotBranched[t]+VarNum])
                    dd[(indx)*ConstNum+j,NotBranched[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<lenNotbranched-2 :
                    for p in range(lenNotbranched-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<lenNotbranched-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(lenNotbranched-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
#     print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
#                 print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
    
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [3]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables): 
    BranchedLen=len(BranchedVariables)
    unbranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            unbranched.append(i)  
    LeUnbranched=len(unbranched)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LeUnbranched*((2 * m) + (4 * n)),1)))

    for i in range (LeUnbranched):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (8):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
#     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [4]:
class MyCut(CPX_CB.UserCutCallback):

    def __call__(self):
         
        self.times_called += 1
        print  'self.times_called', self.times_called
        branched=list()
        values=list()
        branchedNames=list()
        global BranchedMatrix
        senses= "L"
#         print 'self.get_lower_bounds(s)',self.get_lower_bounds()
#         print 'self.get_upper_bounds(s)',self.get_upper_bounds()
#         print 'objective value', self.get_objective_value()
#         print 'values', self.get_values()
        print 'get_num_nodes', self.get_num_nodes()

        lb=self.get_lower_bounds()
        ub=self.get_upper_bounds()               
        for j in range(len(lb)):
            if lb[j]==ub[j]:
                branched.append(j)
                values.append(lb[j])
                branchedNames.append(VarsName[j])
                
        print 'values',values
        print 'branchedNames',branchedNames
        print 'branched',branched

#         if len(values)>0:  
#             BranchedMatrix.append([branched,values])
# #             BranchedValue.append(values)
# #             print 'BranchedMatrix',BranchedMatrix

#             if [BranchedMatrix[-1]]!=[BranchedMatrix[-2]] :
#                 SecModelParameterOutput=SecModelParameter(values, self.m, self.n, self.my_rhs, self.my_coef,branched)
#                 SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],SecModelParameterOutput[2])
#                 print 'SecondModelOutput[1]',SecondModelOutput[1]
#                 print 'SecondModelOutput[0]',SecondModelOutput[0]
#                 if SecondModelOutput[1]>0.000001:
#     #                 print 'bayad cut bezane'
#                     output=GeneratintCutParameters (self.m, self.n, self.my_rhs, self.my_coef,branched, SecondModelOutput[0])
#                     print 'output',output[0]
#                     print 'output[1]',output[1]
#                     lhs=[CPX.SparsePair(ind =branchedNames, val = output[0])]
#                     print 'lhs',lhs[0]
#                     self.add(cut=lhs[0],rhs=output[1][0],sense=senses)
#                     print 'cut added'



In [5]:
def admipex3(c):
    
    c.parameters.preprocessing.presolve = 0

    c.parameters.preprocessing.reduce = 0

    c.parameters.preprocessing.linear.set(0)

    c.parameters.preprocessing.numpass = 0

    c.parameters.mip.strategy.presolvenode = -1

    c.parameters.preprocessing.linear = 0
    
def CutCplex(c):
    c.parameters.mip.cuts.disjunctive.set(-1)
    c.parameters.mip.cuts.bqp.set(-1)
    c.parameters.mip.cuts.cliques.set(-1)
    c.parameters.mip.cuts.covers.set(-1)
    c.parameters.mip.cuts.flowcovers.set(-1)
    c.parameters.mip.cuts.gomory.set(-1)
    c.parameters.mip.cuts.gubcovers.set(-1)
    c.parameters.mip.cuts.implied.set(-1)
    c.parameters.mip.cuts.liftproj.set(-1)
    c.parameters.mip.cuts.localimplied.set(-1)
    c.parameters.mip.cuts.mcfcut.set(-1)
    c.parameters.mip.cuts.mircut.set(-1)
    c.parameters.mip.cuts.zerohalfcut.set(-1)
    c.parameters.mip.cuts.pathcut.set(-1)
    
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))

# admipex3(c)
CutCplex(c)
VarsName=c.variables.get_names()
c.set_log_stream(sys.stdout)
c.set_results_stream(sys.stdout)

cutinst=c.register_callback(MyCut)
cutinst.times_called = 0
cutinst.m=m
cutinst.n=n
cutinst.my_rhs=my_rhs
cutinst.my_coef=my_coef
cutinst.VarsName=VarsName
BranchedMatrix=[[[-1], [-0.0]]]
c.parameters.mip.interval.set(1)
c.parameters.preprocessing.linear=0
c.parameters.mip.strategy.search.set(c.parameters.mip.strategy.search.values.traditional)
c.solve()

c.write("lpex1.lp")
solution = c.solution

# the following line prints the corresponding string
print solution.status[solution.get_status()]
print "Objective value = " , solution.get_objective_value()
print
x = solution.get_values(0, c.variables.get_num()-1)
for j in range(c.variables.get_num()):
    if fabs(x[j]) > 1.0e-10:
        print "Column %d: Value = %17.10g" % (j, x[j])

print "Branch callback was called ", cutinst.times_called, "times"


CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Cuts_Cliques                        -1
CPXPARAM_MIP_Cuts_Covers                         -1
CPXPARAM_MIP_Interval                            1
CPXPARAM_MIP_Cuts_FlowCovers                     -1
CPXPARAM_MIP_Cuts_Implied                        -1
CPXPARAM_MIP_Cuts_GUBCovers                      -1
CPXPARAM_MIP_Cuts_Gomory                         -1
CPXPARAM_MIP_Cuts_PathCut                        -1
CPXPARAM_MIP_Cuts_MIRCut                         -1
CPXPARAM_MIP_Cuts_Disjunctive                    -1
CPXPARAM_MIP_Strategy_Search                     1
CPXPARAM_MIP_Cuts_ZeroHalfCut                    -1
CPXPARAM_MIP_Cuts_MCFCut                         -1
CPXPARAM_MIP_Cuts_LiftProj                       -1
CPXPARAM_MIP_Cuts_LocalImplied                   -1
CPXPARAM_MIP_Cuts_BQP                            -1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 25 rows, 25 col

     14    15      726.6025     5      624.0000      832.6704       67   33.44%             x24 U     14     13     11
     15    14    infeasible            624.0000      832.6704       67   33.44%              x2 U     15     14     12
self.times_called 31
get_num_nodes 16
values [1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x4', 'x5', 'x6', 'x10', 'x12', 'x13', 'x14', 'x16', 'x17', 'x18', 'x20', 'x21', 'x23', 'x24']
branched [0, 1, 2, 4, 5, 6, 10, 12, 13, 14, 16, 17, 18, 20, 21, 23, 24]
self.times_called 32
get_num_nodes 16
values [1.0, 1.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x4', 'x5', 'x6', 'x10', 'x12', 'x13', 'x14', 'x16', 'x17', 'x18', 'x20', 'x21', 'x23', 'x24']
branched [0, 1, 2, 4, 5, 6, 10, 12, 13, 14, 16, 17, 18, 20, 21, 23, 24]
     16    15      698.4500     5      624.0000      832.6704       71   33.44%   

     33    19      827.7153     7      624.0000      827.1060      123   32.55%              x6 U     33      1      2
self.times_called 57
get_num_nodes 34
values [1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10']
branched [1, 6, 10]
self.times_called 58
get_num_nodes 34
values [1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10']
branched [1, 6, 10]
     34    20      779.8807     8      624.0000      827.1060      130   32.55%              x1 U     34     33      3
self.times_called 59
get_num_nodes 35
values [-0.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10']
branched [1, 6, 10]
self.times_called 60
get_num_nodes 35
values [-0.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10']
branched [1, 6, 10]
     35    21      826.1607     8      624.0000      824.7696      132   32.17%              x1 D     35     33      3
self.times_called 61
get_num_nodes 36
values [-0.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10', 'x24']
branched [1, 6, 10, 24]
self.times_called 62
get_num_nodes 36
values [-0.

     49    29    infeasible            624.0000      821.0629      155   31.58%             x13 U     49     48     16
self.times_called 85
get_num_nodes 50
values [1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x6', 'x10', 'x24']
branched [0, 1, 6, 10, 24]
self.times_called 86
get_num_nodes 50
values [1.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x6', 'x10', 'x24']
branched [0, 1, 6, 10, 24]
     50    30      809.7733     6      624.0000      819.2895      160   31.30%              x0 U     50     36      5
Elapsed time = 0.83 sec. (7.15 ticks, tree = 0.01 MB, solutions = 2)
self.times_called 87
get_num_nodes 51
values [1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x6', 'x10', 'x13', 'x24']
branched [0, 1, 6, 10, 13, 24]
self.times_called 88
get_num_nodes 51
values [1.0, -0.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x6', 'x10', 'x13', 'x24']
branched [0, 1, 6, 10, 13, 24]
     51    31      793.5140     7      624.0000      819.2895      168   31.3

     67    41      688.1858     5      624.0000      817.2614      212   30.97%              x0 D     67     66     12
self.times_called 115
get_num_nodes 68
values [-0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0]
branchedNames ['x0', 'x1', 'x2', 'x4', 'x6', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x17', 'x20', 'x21', 'x22', 'x23', 'x24']
branched [0, 1, 2, 4, 6, 9, 10, 11, 12, 13, 14, 15, 17, 20, 21, 22, 23, 24]
self.times_called 116
get_num_nodes 68
values [-0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0]
branchedNames ['x0', 'x1', 'x2', 'x4', 'x6', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x17', 'x20', 'x21', 'x22', 'x23', 'x24']
branched [0, 1, 2, 4, 6, 9, 10, 11, 12, 13, 14, 15, 17, 20, 21, 22, 23, 24]
     68    42      686.9889     5      624.0000      817.2614      213   30.97%             x20 U     68     67     13
self.times_called 117
get_num_nodes 69
values 

...

    475    97      734.7797     3      730.0000      764.3698     1422    4.71%              x0 D    475    156      9
    476    96        cutoff            730.0000      763.5692     1424    4.60%             x12 U    476    407      8
    477    95        cutoff            730.0000      763.3588     1427    4.57%             x24 D    477    189     10
self.times_called 751
get_num_nodes 478
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10', 'x17', 'x20', 'x22']
branched [1, 6, 10, 17, 20, 22]
self.times_called 752
get_num_nodes 478
values [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10', 'x17', 'x20', 'x22']
branched [1, 6, 10, 17, 20, 22]
    478    94      737.0655     6      730.0000      763.0916     1430    4.53%             x22 U    478    390      5
    479    93        cutoff            730.0000      762.7892     1433    4.49%             x22 D    479    212      8
    480    92        cutoff            730.0000      762.5817     1435    4

values [-0.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x6', 'x10', 'x13', 'x19', 'x21', 'x23', 'x24']
branched [0, 1, 6, 10, 13, 19, 21, 23, 24]
    499    87      735.5366     5      730.0000      758.9415     1489    3.96%             x13 D    499    196      9
    500    86    infeasible            730.0000      758.9415     1489    3.96%              x5 D    500    499     10
Elapsed time = 6.80 sec. (27.06 ticks, tree = 0.02 MB, solutions = 6)
    501    85        cutoff            730.0000      758.2536     1492    3.87%             x24 D    501    433      7
    502    84        cutoff            730.0000      758.1021     1494    3.85%             x22 U    502    135      4
    503    83        cutoff            730.0000      758.0094     1496    3.84%             x22 D    503    270      9
self.times_called 773
get_num_nodes 504
values [-0.0, 1.0, 1.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x4', 'x5', 'x6', 'x10', 'x13', 'x1

values [1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0, 1.0]
branchedNames ['x0', 'x1', 'x2', 'x5', 'x6', 'x10', 'x14', 'x16', 'x20', 'x24']
branched [0, 1, 2, 5, 6, 10, 14, 16, 20, 24]
    528    74      744.6592     7      730.0000      752.9236     1562    3.14%              x2 U    528     92      8
    529    73    infeasible            730.0000      752.9236     1562    3.14%             x11 D    529    528      9
    530    72    infeasible            730.0000      752.9236     1562    3.14%             x11 U    530    528      9
Elapsed time = 7.09 sec. (27.90 ticks, tree = 0.02 MB, solutions = 6)
    531    71        cutoff            730.0000      752.4717     1565    3.08%              x7 U    531    436     10
self.times_called 795
get_num_nodes 532
values [-0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0, -0.0, -0.0, 1.0]
branchedNames ['x0', 'x1', 'x6', 'x7', 'x10', 'x13', 'x15', 'x19', 'x21', 'x23', 'x24']
branched [0, 1, 6, 7, 10, 13, 15, 19, 21, 23, 24]
self.times_called 79

    551    57        cutoff            730.0000      747.5131     1620    2.40%             x22 D    551    509      6
    552    56        cutoff            730.0000      747.4535     1623    2.39%             x13 D    552    392      7
self.times_called 811
get_num_nodes 553
values [-0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10', 'x13', 'x14', 'x17', 'x22']
branched [1, 6, 10, 13, 14, 17, 22]
self.times_called 812
get_num_nodes 553
values [-0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 1.0]
branchedNames ['x1', 'x6', 'x10', 'x13', 'x14', 'x17', 'x22']
branched [1, 6, 10, 13, 14, 17, 22]
    553    57      744.7548     6      730.0000      747.4358     1626    2.39%             x14 U    553    220      6
    554    56        cutoff            730.0000      747.4358     1627    2.39%             x24 U    554    553      7
    555    55        cutoff            730.0000      747.4358     1629    2.39%             x24 D    555    553      7
    556    54        cutoff        

    584    31      731.8813     5      730.0000      741.1278     1679    1.52%             x18 D    584     81     11
    585    30        cutoff            730.0000      740.6712     1682    1.46%             x22 D    585    465      8
    586    29        cutoff            730.0000      740.3634     1685    1.42%             x23 U    586    279      7
    587    28    infeasible            730.0000      740.2351     1685    1.40%             x22 D    587    382      6
    588    27        cutoff            730.0000      739.6361     1688    1.32%             x23 U    588    140      8
    589    26        cutoff            730.0000      739.5790     1690    1.31%             x22 U    589    382      6
    590    25        cutoff            730.0000      739.5306     1692    1.31%             x17 D    590     94      9
Elapsed time = 7.70 sec. (32.67 ticks, tree = 0.01 MB, solutions = 6)
    591    24        cutoff            730.0000      738.8562     1694    1.21%              x4 U